In [3]:
from torch_geometric.datasets import OGB_MAG

dataset = OGB_MAG(root='./data', preprocess='metapath2vec')
data = dataset[0]

In [4]:
data

HeteroData(
  paper={
    x=[736389, 128],
    year=[736389],
    y=[736389],
    train_mask=[736389],
    val_mask=[736389],
    test_mask=[736389]
  },
  author={ x=[1134649, 128] },
  institution={ x=[8740, 128] },
  field_of_study={ x=[59965, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 1043998] },
  (author, writes, paper)={ edge_index=[2, 7145660] },
  (paper, cites, paper)={ edge_index=[2, 5416271] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 7505078] }
)

In [47]:
import os
import networkx as nx
import json
from tqdm import tqdm
import torch
import torch_geometric

def nx2hetero(G):
	ids_by_type = {
		"playlist": {},
		"track": {},
		"artist": {},
		"album": {}
	}
	def node_id(type, id, exception=False):
		d = ids_by_type[type]
		if id not in d:
			if exception:
				raise Exception(f'node {id} not found for type {type}')
			d[id] = len(d)
		return d[id]

	
	node_features_by_type = {
		"playlist": [],
		"track": [],
		"artist": [],
		"album": []
	}
	for node in G.nodes(data=True):
		t = node[1]["node_type"]
		if t == "playlist":
			node_id("playlist", node[0])
			node_features_by_type["playlist"] += [node[1]["num_tracks"]]
		elif t == "track":
			node_id("track", node[0])
			node_features_by_type["track"] += [node[1]["num_albums"]]
		elif t == "artist":
			node_id("artist", node[0])
			node_features_by_type["artist"] += [node[1]["num_tracks"]]
		elif t == "album":
			node_id("album", node[0])
			node_features_by_type["album"] += [node[1]["num_tracks"]]

	edge_index_by_type = {
		("playlist", "contains", "track"): [],
		("album", "includes", "track"): [],
		("artist", "authors", "track"): []
	}
	for edge in G.edges(data=True):
		if G[edge[0]][edge[1]]["edge_type"] == "track-playlist":
			s_id = node_id("track", edge[1], exception=True)
			t_id = node_id("playlist", edge[0], exception=True)
			
			edge_index_by_type[("playlist", "contains", "track")] += [(t_id, s_id)]
		elif G[edge[0]][edge[1]]["edge_type"] == "track-album":
			s_id = node_id("track", edge[0], exception=True)
			t_id = node_id("album", edge[1], exception=True)
			
			edge_index_by_type[("album", "includes", "track")] += [(t_id, s_id)]
		elif G[edge[0]][edge[1]]["edge_type"] == "track-artist":
			s_id = node_id("track", edge[0], exception=True)
			t_id = node_id("artist", edge[1], exception=True)
			
			edge_index_by_type[("artist", "authors", "track")] += [(t_id, s_id)]

	# construct HeteroData
	hetero = torch_geometric.data.HeteroData()

	# add initial node features
	hetero["playlist"].x = torch.IntTensor(node_features_by_type["playlist"]).reshape(-1,1)
	hetero["track"].x = torch.IntTensor(node_features_by_type["track"]).reshape(-1,1)
	hetero["artist"].x = torch.IntTensor(node_features_by_type["artist"]).reshape(-1,1)
	hetero["album"].x = torch.IntTensor(node_features_by_type["album"]).reshape(-1,1)

	# add edge indices
	hetero["playlist", "contains", "track"].edge_index = torch.tensor(edge_index_by_type[("playlist", "contains", "track")]).t()
	hetero["album", "includes", "track"].edge_index = torch.tensor(edge_index_by_type[("album", "includes", "track")]).t()
	hetero["artist", "authors", "track"].edge_index = torch.tensor(edge_index_by_type[("artist", "authors", "track")]).t()

	return hetero


In [48]:
import pickle
base = "spotify_million_playlist_dataset"
pickles = base + "/pickles"
graph_path = os.path.join(pickles, "G_example.pkl")

G = pickle.load(open(graph_path, "rb"))

our_data = nx2hetero(G)

{'spotify:playlist:42395', 'spotify:playlist:42582', 'spotify:playlist:42412', 'spotify:playlist:42302', 'spotify:playlist:42522', 'spotify:playlist:42601', 'spotify:playlist:42707', 'spotify:playlist:42839', 'spotify:playlist:42741', 'spotify:playlist:42802', 'spotify:playlist:42010', 'spotify:playlist:42136', 'spotify:playlist:42208', 'spotify:playlist:42900', 'spotify:playlist:42527', 'spotify:playlist:42644', 'spotify:playlist:42983', 'spotify:playlist:42283', 'spotify:playlist:42356', 'spotify:playlist:42161', 'spotify:playlist:42114', 'spotify:playlist:42789', 'spotify:playlist:42476', 'spotify:playlist:42876', 'spotify:playlist:42426', 'spotify:playlist:42158', 'spotify:playlist:42525', 'spotify:playlist:42126', 'spotify:playlist:42274', 'spotify:playlist:42298', 'spotify:playlist:42791', 'spotify:playlist:42927', 'spotify:playlist:42182', 'spotify:playlist:42581', 'spotify:playlist:42932', 'spotify:playlist:42716', 'spotify:playlist:42729', 'spotify:playlist:42203', 'spotify:pl

Exception: playlist not found

In [35]:
# create training mask for playlist nodes
train_mask = torch.zeros(our_data["playlist"].x.shape[0], dtype=torch.bool)
train_mask[torch.randperm(train_mask.shape[0])[:int(train_mask.shape[0]*0.8)]] = True

our_data["playlist"].train_mask = train_mask

In [36]:
our_data["playlist", "contains", "track"].edge_index

tensor([[    0,     0,     0,  ..., 35668, 35668, 35668],
        [    0,     1,     2,  ..., 36226, 36227, 36228]])

In [20]:
our_data.metadata()

(['playlist', 'track', 'artist', 'album'],
 [('playlist', 'contains', 'track'),
  ('album', 'includes', 'track'),
  ('artist', 'authors', 'track'),
  ('track', 'rev_contains', 'playlist'),
  ('track', 'rev_includes', 'album'),
  ('track', 'rev_authors', 'artist')])

In [15]:
data = our_data

In [16]:
data.is_undirected()

False

In [17]:
homogeneous_data = data.to_homogeneous()
homogeneous_data

Data(edge_index=[2, 136909], x=[66849, 1], train_mask=[66849], node_type=[66849], edge_type=[136909])

In [18]:
import torch_geometric.transforms as T

data = T.ToUndirected()(data)
# data = T.NormalizeFeatures()(data)

In [19]:
data.is_undirected()

True

In [21]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.nn import SAGEConv, to_hetero
import torch
import torch.nn.functional as F

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


model = GNN(hidden_channels=64, out_channels=dataset.num_classes)
model = to_hetero(model, data.metadata(), aggr='sum')
# model = model.to('cuda:0')

In [22]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [23]:
data.validate()

ValueError: 'edge_index' of edge type ('playlist', 'contains', 'track') contains larger source indices than the number of nodes (1000) of this node type in 'HeteroData' (found 66694)

In [ ]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.loader import NeighborLoader

# transform = T.ToUndirected()  # Add reverse edge types.
# data = OGB_MAG(root='./data', preprocess='metapath2vec', transform=transform)[0]

train_loader = NeighborLoader(
    data,
    # Sample 15 neighbors for each node and each edge type for 2 iterations:
    num_neighbors=[15] * 2,
    # num_neighbors = {key: [15] * 2 for key in data.edge_types} will sample different ammounts for each node type
    # Use a batch size of 128 for sampling training nodes of type "paper":
    batch_size=128,
    input_nodes=('playlist', data['playlist'].train_mask),
)

batch = next(iter(train_loader))

: 

: 

In [ ]:
batch

HeteroData(
  paper={
    x=[20796, 128],
    year=[20796],
    y=[20796],
    train_mask=[20796],
    val_mask=[20796],
    test_mask=[20796],
    input_id=[128],
    batch_size=128
  },
  author={ x=[4454, 128] },
  institution={ x=[306, 128] },
  field_of_study={ x=[2584, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 0] },
  (author, writes, paper)={ edge_index=[2, 5916] },
  (paper, cites, paper)={ edge_index=[2, 11751] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 10573] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 821] },
  (paper, rev_writes, author)={ edge_index=[2, 5484] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 10432] }
)

In [ ]:
def train():
    model.train()

    total_examples = total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        # batch = batch.to('cuda:0')
        batch_size = batch['paper'].batch_size
        out = model(batch.x_dict, batch.edge_index_dict)
        loss = F.cross_entropy(out['paper'][:batch_size],
                               batch['paper'].y[:batch_size])
        loss.backward()
        optimizer.step()

        total_examples += batch_size
        print(f'Loss: {loss:.4f}')
        total_loss += float(loss) * batch_size

    return total_loss / total_examples

In [ ]:
train()

ValueError: `MessagePassing.propagate` only supports integer tensors of shape `[2, num_messages]`, `torch_sparse.SparseTensor` or `torch.sparse.Tensor` for argument `edge_index`.